In [19]:
import warnings
warnings.filterwarnings("ignore")
from evaluation.mol_structure import list_of_smiles_to_nx_graphs
from evaluation.mol_structure import draw_one_mol, draw_graphs
from evaluation.new_structural_metric import symmetric_graph_set_distance,atom,cycle,neighborhood
from evaluate import evaluate
#import utils.graph_generators as gen
import torch
import networkx as nx
import os 
import sys
import json
current = os.getcwd()
parent = os.path.dirname(current)
sys.path.append(parent)
import rdkit
import numpy as np
from evaluation.utils import get_data,get_graph_data,  get_mock_data,remove_empty_graphs_and_targets,\
    get_generated_data,preprocess,get_generated_graph_data,_preprocess



# Rewired

In [2]:
import pandas as pd
def get_rewired_graph_data(name,percent):
    generated_neg_graphs=pd.read_pickle(f'data/rewired/{name}_train2_neg/{name}_train2_neg_rewired_with_{percent}.pkl')
    generated_pos_graphs=pd.read_pickle(f'data/rewired/{name}_train2_pos/{name}_train2_pos_rewired_with_{percent}.pkl')
    generated_graphs,generated_targets = generated_pos_graphs + generated_neg_graphs, [1]*len(generated_pos_graphs)+[0]*len(generated_neg_graphs)
    return  generated_graphs, np.array(generated_targets)

In [3]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn','structural', 'auc_roc']
generators=['swingnn','gdss']
splits=['_50']
rewiring_ratio=['5','10','25','33','40','50','66','75','90','100']
metrics={}
for generator in ['rewired']:
    for dataset_name in ['synthetic_1','synthetic_2']:
        for split in splits:
            for percent in rewiring_ratio:
                print(dataset_name+split)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_rewired_graph_data(dataset_name+split,percent)
                
                splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                metrics=evaluate(train1_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)

synthetic_1_50
Now computing classifier based metrics
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Now computing structural based metrics
Now computing the auc_roc based  metric
0.7620000000000001 0.8126 0.7162 0.775
{'activations_time': 0.7384419441223145, 'fid': 0.19760020608472928, 'fid_time': 1.456873893737793, 'kid': 0.001262784, 'kid_time': 7.389859914779663, 'precision': 0.9075, 'recall': 0.985, 'f1_pr': 0.9446731607591992, 'precision_time': 1.6914477348327637, 'recall_time': 1.6914477348327637, 'f1_pr_time': 1.6914477348327637, 'density': 0.891, 'coverage': 0.9775, 'f1_dc': 0.9322578137777491, 'density_time': 0.9975109100341797, 'coverage_time': 0.9975109100341797, 'f1_dc_time': 0.9975109100341797, 'mmd_rbf': 0.004877497674897313, 'mmd_rbf_time': 1.1372909545898438, 'mmd_linear': 0.15119714

# Pertubed with ER

In [4]:
import pandas as pd
def get_corrupted_graph_data(name,percent):
    generated_neg_graphs=pd.read_pickle(f'data/corrupted/{name}_train2_neg/{name}_train2_neg_perturbed_with_{percent}.pkl')
    generated_pos_graphs=pd.read_pickle(f'data/corrupted/{name}_train2_pos/{name}_train2_pos_perturbed_with_{percent}.pkl')
    generated_graphs,generated_targets = generated_pos_graphs + generated_neg_graphs, [1]*len(generated_pos_graphs)+[0]*len(generated_neg_graphs)
    return  generated_graphs, np.array(generated_targets)

In [5]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn', 'structural', 'auc_roc']
generators=['swingnn','gdss']
splits=['_50']
rewiring_ratio=['5','10','25','33','40','50','66','75','90','100']
metrics={}
for generator in ['corrupted']:
    for dataset_name in ['synthetic_1','synthetic_1']:
        for split in splits:
            for percent in rewiring_ratio:
                print(dataset_name+split)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_corrupted_graph_data(dataset_name+split,percent)
                splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                metrics=evaluate(train1_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)

synthetic_1_50
Now computing classifier based metrics
Now computing structural based metrics
Now computing the auc_roc based  metric
0.7620000000000001 0.8126 0.7712500000000001 0.81425
{'activations_time': 0.038542985916137695, 'fid': 0.07721535721607609, 'fid_time': 0.06975126266479492, 'kid': -0.0064834356, 'kid_time': 0.07900714874267578, 'precision': 0.98, 'recall': 0.9675, 'f1_pr': 0.9737198848792309, 'precision_time': 0.19340276718139648, 'recall_time': 0.19340276718139648, 'f1_pr_time': 0.19340276718139648, 'density': 0.969, 'coverage': 0.9525, 'f1_dc': 0.9606891576460301, 'density_time': 0.10789608955383301, 'coverage_time': 0.10789608955383301, 'f1_dc_time': 0.10789608955383301, 'mmd_rbf': 0.004815790802240372, 'mmd_rbf_time': 0.1655442714691162, 'mmd_linear': 0.001991492, 'mmd_linear_time': 0.0385892391204834, 'wl_mmd': 0.005, 'wl_mmd_time': 0.467540979385376, 'nspdk_mmd': 0.004267183065973235, 'nspdk_mmd_time': 9.159044981002808, 'degree_mmd': 0.00015155927486643073, 'degre

# Mode Dropping


In [6]:
import pandas as pd
def get_mode_dropping_graph_data(name,percent):
    #try:
   generated_graphs=pd.read_pickle(f'data/mode_dropping/{name}/{name}_no_of_clusters_dropped_{percent}.pkl')
   targets=[]
   for i, g in enumerate(generated_graphs):
        target= int(g.nodes[0]['target'])
        targets.append(target)
   return  generated_graphs, np.array(targets)

In [7]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn', 'structural', 'auc_roc']
generators=['swingnn','gdss']
splits=['_50']
rewiring_ratio=[i for i in range(1, 32)]
metrics={}
for generator in ['mode_dropping']:
    for dataset_name in ['synthetic_1']:
        for split in splits:
            for percent in rewiring_ratio:
                print(dataset_name+split)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_mode_dropping_graph_data(dataset_name+split,percent)
                splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                metrics=evaluate(train1_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)



synthetic_1_50
Now computing classifier based metrics
Now computing structural based metrics
Now computing the auc_roc based  metric
0.7620000000000001 0.8126 0.7547999999999999 0.7911000000000001
{'activations_time': 0.04551410675048828, 'fid': 0.026706130864745603, 'fid_time': 0.05444693565368652, 'kid': -0.0063757896, 'kid_time': 0.07775712013244629, 'precision': 0.9725, 'recall': 0.9825, 'f1_pr': 0.9774844248140684, 'precision_time': 0.5935671329498291, 'recall_time': 0.5935671329498291, 'f1_pr_time': 0.5935671329498291, 'density': 1.006, 'coverage': 0.97, 'f1_dc': 0.9876820680964766, 'density_time': 0.32853007316589355, 'coverage_time': 0.32853007316589355, 'f1_dc_time': 0.32853007316589355, 'mmd_rbf': 0.006175786256790161, 'mmd_rbf_time': 0.524177074432373, 'mmd_linear': 0.020827923, 'mmd_linear_time': 0.04556608200073242, 'wl_mmd': 0.005, 'wl_mmd_time': 0.5297548770904541, 'nspdk_mmd': 0.004323386956742337, 'nspdk_mmd_time': 11.450646162033081, 'degree_mmd': 0.000146943747335592

In [8]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn', 'structural', 'auc_roc']
generators=['swingnn','gdss']
splits=['_50']
rewiring_ratio=[i for i in range(1, 33)]
metrics={}
for generator in ['mode_dropping']:
    for dataset_name in ['synthetic_2']:
        for split in splits:
            for percent in rewiring_ratio:
                print(dataset_name+split)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_mode_dropping_graph_data(dataset_name+split,percent)
                splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                metrics=evaluate(train1_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)

synthetic_2_50
Now computing classifier based metrics
Now computing structural based metrics
Now computing the auc_roc based  metric
0.8065000000000001 0.82805 0.81105 0.8306000000000001
{'activations_time': 0.04191279411315918, 'fid': 0.4673616323057246, 'fid_time': 0.06281471252441406, 'kid': 0.02029562, 'kid_time': 0.07401895523071289, 'precision': 0.965, 'recall': 0.955, 'f1_pr': 0.9599839586045978, 'precision_time': 0.13985681533813477, 'recall_time': 0.13985681533813477, 'f1_pr_time': 0.13985681533813477, 'density': 0.9595000000000001, 'coverage': 0.97, 'f1_dc': 0.9647314307185207, 'density_time': 0.13818073272705078, 'coverage_time': 0.13818073272705078, 'f1_dc_time': 0.13818073272705078, 'mmd_rbf': 0.008294671773910522, 'mmd_rbf_time': 0.1538848876953125, 'mmd_linear': 0.28981638, 'mmd_linear_time': 0.04195761680603027, 'wl_mmd': 0.005, 'wl_mmd_time': 0.46073079109191895, 'nspdk_mmd': 0.003759602999053646, 'nspdk_mmd_time': 8.975366115570068, 'degree_mmd': 6.0964231288318516e-0

# Mode Collapse

In [9]:

import pandas as pd
def get_mode_collapse_graph_data(name,percent):
    #try:
   generated_graphs=pd.read_pickle(f'data/mode_collapse/{name}/{name}_no_of_clusters_collapsed_{percent}.pkl')
   targets=[]
   for i, g in enumerate(generated_graphs):
        target= int(g.nodes[0]['target'])
        targets.append(target)
   return  generated_graphs, np.array(targets)

In [10]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn', 'structural', 'auc_roc']
generators=['swingnn','gdss']
splits=['_50']
rewiring_ratio=[i for i in range(1, 32)]
metrics={}
for generator in ['mode_collapse']:
    for dataset_name in ['synthetic_1']:
        for split in splits:
            for percent in rewiring_ratio:
                print(dataset_name+split)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_mode_collapse_graph_data(dataset_name+split,percent)
                splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                metrics=evaluate(train1_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)

synthetic_1_50
Now computing classifier based metrics
Now computing structural based metrics
Now computing the auc_roc based  metric
0.7620000000000001 0.8126 0.7398 0.7906
{'activations_time': 0.04692196846008301, 'fid': 0.1320485407885741, 'fid_time': 0.05339813232421875, 'kid': -0.0021275282, 'kid_time': 0.06524181365966797, 'precision': 0.99, 'recall': 0.975, 'f1_pr': 0.9824527486743136, 'precision_time': 0.23654770851135254, 'recall_time': 0.23654770851135254, 'f1_pr_time': 0.23654770851135254, 'density': 1.0475, 'coverage': 0.98, 'f1_dc': 1.0126363982599431, 'density_time': 0.17254996299743652, 'coverage_time': 0.17254996299743652, 'f1_dc_time': 0.17254996299743652, 'mmd_rbf': 0.0057850182056427, 'mmd_rbf_time': 0.1793229579925537, 'mmd_linear': 0.0812089, 'mmd_linear_time': 0.04696917533874512, 'wl_mmd': 0.005, 'wl_mmd_time': 0.5076248645782471, 'nspdk_mmd': 0.004368843089490648, 'nspdk_mmd_time': 9.416892051696777, 'degree_mmd': 0.00012062901060772013, 'degree_mmd_time': 7.9832

In [11]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn', 'structural', 'auc_roc']
generators=['swingnn','gdss']
splits=['_50']
rewiring_ratio=[i for i in range(1, 33)]
metrics={}
for generator in ['mode_collapse']:
    for dataset_name in ['synthetic_2']:
        for split in splits:
            for percent in rewiring_ratio:
                print(dataset_name+split)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_mode_collapse_graph_data(dataset_name+split,percent)
                splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                metrics=evaluate(train1_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['degree_of_perturbation']=percent
                print(metrics)

synthetic_2_50
Now computing classifier based metrics
Now computing structural based metrics
Now computing the auc_roc based  metric
0.8065000000000001 0.82805 0.80315 0.8556500000000001
{'activations_time': 0.04502081871032715, 'fid': 0.5743247424700542, 'fid_time': 0.08963394165039062, 'kid': 0.01957488, 'kid_time': 0.06555962562561035, 'precision': 0.975, 'recall': 0.97, 'f1_pr': 0.9725035733308653, 'precision_time': 0.19227361679077148, 'recall_time': 0.19227361679077148, 'f1_pr_time': 0.19227361679077148, 'density': 1.066, 'coverage': 0.955, 'f1_dc': 1.0074617867215565, 'density_time': 0.11194682121276855, 'coverage_time': 0.11194682121276855, 'f1_dc_time': 0.11194682121276855, 'mmd_rbf': 0.005675438791513443, 'mmd_rbf_time': 0.14234685897827148, 'mmd_linear': 0.3699737, 'mmd_linear_time': 0.045076847076416016, 'wl_mmd': 0.005, 'wl_mmd_time': 0.4935731887817383, 'nspdk_mmd': 0.0040525563426479505, 'nspdk_mmd_time': 9.536576986312866, 'degree_mmd': 7.758755215681035e-05, 'degree_mm

# Additionally train1 versus train2

In [12]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn', 'structural', 'auc_roc']
generators=['swingnn','gdss']
splits=['_50']
rewiring_ratio=[i for i in range(1, 33)]
metrics={}
for generator in ['train2']:
    for dataset_name in ['synthetic_1','synthetic_2']:
        for split in splits:
            print(dataset_name+split)
            train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
            generated_graphs, generated_targets= train2_graphs, train2_targets
            splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
            metrics=evaluate(train1_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
            metrics['dataset']=dataset_name+split
            metrics['generator_name']=generator
            metrics['degree_of_perturbation']=percent
            print(metrics)

synthetic_1_50
Now computing classifier based metrics
Now computing structural based metrics
Now computing the auc_roc based  metric
0.7620000000000001 0.8126 0.73925 0.8126
{'activations_time': 0.03923916816711426, 'fid': 0.04679155621931841, 'fid_time': 0.04503011703491211, 'kid': -0.0033990145, 'kid_time': 0.07478213310241699, 'precision': 0.975, 'recall': 0.9775, 'f1_pr': 0.9762583995042304, 'precision_time': 0.20879530906677246, 'recall_time': 0.20879530906677246, 'f1_pr_time': 0.20879530906677246, 'density': 0.9960000000000001, 'coverage': 0.9, 'f1_dc': 0.9455796458898114, 'density_time': 0.13438010215759277, 'coverage_time': 0.13438010215759277, 'f1_dc_time': 0.13438010215759277, 'mmd_rbf': 0.006395334377884865, 'mmd_rbf_time': 0.1645030975341797, 'mmd_linear': 0.03953388, 'mmd_linear_time': 0.0392911434173584, 'wl_mmd': 0.005, 'wl_mmd_time': 0.46033692359924316, 'nspdk_mmd': 0.004269677911791503, 'nspdk_mmd_time': 9.157461881637573, 'degree_mmd': 0.00014829539083982546, 'degree

# SWINGNN as we increase the number of training epochs

In [16]:

import pandas as pd
from glob import glob
from sklearn.utils import shuffle                                                                                                                                                                                                                                                                                                                                                                                                                                                              
from evaluation.mol_structure import list_of_smiles_to_nx_graphs
from evaluation.utils import remove_empty_graphs_and_targets
import numpy as np
def get_swingnn_graph_data(name,percent,split,path='data/SWINGNN_experiments/'):
     path+=f'{percent}-percent/'
     path_postives=path+r'{}/{}/pos/*.pkl'.format(name,name+split)
     files = glob(path_postives)
     generated_pos_graphs=pd.read_pickle(files[0])
     path_negatives=path+r'{}/{}/neg/*.pkl'.format(name,name+split)
     files = glob(path_negatives)
     generated_neg_graphs=pd.read_pickle(files[0])
     generated_graphs,generated_targets = generated_pos_graphs + generated_neg_graphs, [1]*len(generated_pos_graphs)+[0]*len(generated_neg_graphs)

     return  generated_graphs, np.array(generated_targets)

In [14]:
name='synthetic_1'
percent='100'
split='_50'
get_swingnn_graph_data(name,percent,split)

([<networkx.classes.graph.Graph at 0x336a24d30>,
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [3]:
from glob import glob
from sklearn.utils import shuffle                                                                                                                                                                                                                                                                                                                                                                                                                                                              
from evaluation.mol_structure import list_of_smiles_to_nx_graphs
from evaluation.utils import remove_empty_graphs_and_targets
import numpy as np

def get_swingnn_molecular_data(name,percent, path='data/SWINGNN_experiments/', generator_name='stgg',return_smiles=False):
     path+=f'{percent}-percent/'
     pos_list, neg_list=[],[]
     path_postives=path+r'{}/{}/pos/*.txt'.format(name,name+'_50')
     files = glob(path_postives)
     
     with open(files[0]) as my_file:
          for line in my_file:
               pos_list.append(line.strip())

     path_negatives=path+r'{}/{}/neg/*.txt'.format(name,name+'_50')
     files = glob(path_negatives)
     
     with open(files[0]) as my_file:
          for line in my_file:
               neg_list.append(line.strip())

     pos_graphs=list_of_smiles_to_nx_graphs(pos_list)
     neg_graphs=list_of_smiles_to_nx_graphs(neg_list)
     #print(len(neg_graphs))
     generated_graphs,generated_targets = pos_graphs + neg_graphs, [1]*len(pos_graphs)+[0]*len(neg_graphs)
     #generated_graphs, generated_targets = shuffle(generated_graphs, generated_targets)
     generated_graphs, generated_targets = remove_empty_graphs_and_targets(generated_graphs, generated_targets)
     if return_smiles:
          return  generated_graphs, generated_targets,pos_list+neg_list
     return  generated_graphs, np.array(generated_targets)

In [16]:
name='ames'
percent='100'
split='_50'
_,generated_targets, generated_smiles=get_swingnn_molecular_data(name,percent,return_smiles=True) 

In [10]:
generated_targets[-1]

0

## Molecular graphs

### nn and structural 

In [17]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
splits=['_25','_33','_40','_50']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn','structural']#'molecular']
#leave it blank to compute all the structural structures 
generators=['swingnn']
experiments=[5,10,25,33,40,50,66,75,100]
device=torch.device('cpu')
metrics={}
split='_50'
for generator in generators:
    for dataset_name in current_smiles_datasets:
             for percent in experiments:
                print(dataset_name+split)
                train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
                generated_graphs, generated_targets=get_swingnn_molecular_data(dataset_name,percent)
                splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                metrics=evaluate(train1_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['numer_of_epochs_trained']=(percent*5000)/100
                print(metrics)

ames_50
Now computing classifier based metrics
Now computing structural based metrics
{'activations_time': 0.21366095542907715, 'fid': 26.50553763515596, 'fid_time': 0.21585965156555176, 'kid': 0.9191108, 'kid_time': 0.305006742477417, 'precision': 0.8629760502552022, 'recall': 0.5826462504907735, 'f1_pr': 0.6956412606466655, 'precision_time': 1.2940280437469482, 'recall_time': 1.2940280437469482, 'f1_pr_time': 1.2940280437469482, 'density': 0.7016882606988615, 'coverage': 0.3376521397722811, 'f1_dc': 0.4559283499249382, 'density_time': 0.938338041305542, 'coverage_time': 0.938338041305542, 'f1_dc_time': 0.938338041305542, 'mmd_rbf': 0.18399828672409058, 'mmd_rbf_time': 1.4124457836151123, 'mmd_linear': 16.260548, 'mmd_linear_time': 0.2137889862060547, 'wl_mmd': 0.0007852375343541421, 'wl_mmd_time': 2.1692988872528076, 'nspdk_mmd': 0.08389942961717695, 'nspdk_mmd_time': 12.742777824401855, 'degree_mmd': 0.0228826161270963, 'degree_mmd_time': 37.297131, 'cluster_mmd': 0.0003904567516506

#### train2

In [18]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn','structural']#'molecular']
#leave it blank to compute all the structural structures 
generators=['train2']
experiments=[5,10,25,33,40,50,66,75,100]
device=torch.device('cpu')
metrics={}
split='_50'
for generator in generators:
    for dataset_name in current_smiles_datasets:
        for split in splits:
            print(dataset_name+split)
            train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets  =get_data(dataset_name+split, return_smiles=False)
            generated_graphs, generated_targets=train2_graphs , train2_targets
            splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
            metrics=evaluate(train1_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
            metrics['dataset']=dataset_name+split
            metrics['generator_name']=generator
            print(metrics)

ames_25
Now computing classifier based metrics
Now computing structural based metrics
{'activations_time': 0.09668684005737305, 'fid': 2.9564222480168496, 'fid_time': 0.1282970905303955, 'kid': 0.01800251, 'kid_time': 0.15801072120666504, 'precision': 0.969363707776905, 'recall': 0.9630793401413983, 'f1_pr': 0.9662213055809473, 'precision_time': 0.5455377101898193, 'recall_time': 0.5455377101898193, 'f1_pr_time': 0.5455377101898193, 'density': 1.0227808326787118, 'coverage': 0.9748625294579732, 'f1_dc': 0.9982569686750845, 'density_time': 0.399242639541626, 'coverage_time': 0.399242639541626, 'f1_dc_time': 0.399242639541626, 'mmd_rbf': 0.002381354570388794, 'mmd_rbf_time': 0.5534439086914062, 'mmd_linear': 0.16101262, 'mmd_linear_time': 0.09678387641906738, 'wl_mmd': 0.0015710919088766694, 'wl_mmd_time': 3.6933791637420654, 'nspdk_mmd': 0.0014485484440663887, 'nspdk_mmd_time': 7.593865871429443, 'degree_mmd': 0.00026359444890333705, 'degree_mmd_time': 28.933659, 'cluster_mmd': 7.422813

### molecular

In [23]:
from evaluation.moses.metrics.metrics import get_all_metrics
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
split1='_50'
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn','structural']#'molecular']
#leave it blank to compute all the structural structures 
generators=['swingnn']
experiments=[5,10,25,33,40,50,66,75,100]
device=torch.device('cpu')
metrics={}
for generator in generators:
    for dataset_name in current_smiles_datasets:
            for percent in experiments:
                print(dataset_name+split1)
                graphs, splits =get_data(dataset_name+split1, return_smiles=True)
                train1_smiles=splits['train1_pos_smiles']+splits['train1_neg_smiles'] 
                _,generated_targets, generated_smiles=get_swingnn_molecular_data(name,percent,return_smiles=True) 
                metrics=get_all_metrics(gen=generated_smiles,train=train1_smiles,test=train1_smiles)
                metrics['dataset']=dataset_name+split1
                metrics['generator_name']=generator
                metrics['numer_of_epochs_trained']=(percent*5000)/100 
                print(metrics)

ames_50
{'valid': 1.0, 'unique@1000': 0.992, 'unique@10000': 0.9752650176678446, 'FCD/Test': 17.029497372930688, 'SNN/Test': 0.2827048919975032, 'Frag/Test': 0.16546222270984667, 'Scaf/Test': 0.058704077404745036, 'FCD/TestSF': 37.4445566284182, 'SNN/TestSF': 0.19734603975408443, 'Frag/TestSF': 0.10150397926772925, 'Scaf/TestSF': 4.1041894040527716e-05, 'IntDiv': 0.9190337370237776, 'IntDiv2': 0.9022144090456522, 'Filters': 0.5127601099332548, 'logP': 0.23130422065174702, 'SA': 1.536261045891231, 'QED': 0.07715936494530255, 'weight': 51.08613506085594, 'Novelty': 0.9935587761674718, 'dataset': 'ames_50', 'generator_name': 'swingnn', 'numer_of_epochs_trained': 250.0}
ames_50
{'valid': 1.0, 'unique@1000': 0.987, 'unique@10000': 0.9733019238319591, 'FCD/Test': 15.457292898291747, 'SNN/Test': 0.2953605191276782, 'Frag/Test': 0.21066794374915343, 'Scaf/Test': 0.09902119663282583, 'FCD/TestSF': 35.64221421636759, 'SNN/TestSF': 0.21063697174583457, 'Frag/TestSF': 0.14979314122713538, 'Scaf/Te

#### train2


In [24]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
split ='_50'
device=torch.device('cpu')
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']#'common_substructures']
metrics_type=['nn','structural']#'molecular']
#leave it blank to compute all the structural structures 
generators=['train2']
experiments=[5,10,25,33,40,50,66,75,100]
device=torch.device('cpu')
metrics={}
for generator in generators:
    for dataset_name in current_smiles_datasets:
            print(dataset_name+split)
            graphs, splits =get_data(dataset_name+split, return_smiles=True)
            train1_smiles=splits['train1_pos_smiles']+splits['train1_neg_smiles'] 
            train2_smiles=splits['train2_pos_smiles']+splits['train2_neg_smiles'] 
            metrics=get_all_metrics(gen=train2_smiles,train=train1_smiles,test=train1_smiles)
            metrics['dataset']=dataset_name+split
            metrics['generator_name']=generator
            print(metrics)

ames_50
{'valid': 1.0, 'unique@1000': 0.999, 'unique@10000': 0.9988221436984688, 'FCD/Test': 0.8156919778498093, 'SNN/Test': 0.5756633373852753, 'Frag/Test': 0.9878820451873492, 'Scaf/Test': 0.8593618010840082, 'FCD/TestSF': 22.184284328679503, 'SNN/TestSF': 0.33677946461981123, 'Frag/TestSF': 0.7366064124029432, 'Scaf/TestSF': 0.20333785426573803, 'IntDiv': 0.9102053951183938, 'IntDiv2': 0.8921645781772736, 'Filters': 0.5618374558303887, 'logP': 0.08376508833922258, 'SA': 0.04752492131394497, 'QED': 0.0039007607322941876, 'weight': 4.032424028268554, 'Novelty': 0.9976415094339622, 'dataset': 'ames_50', 'generator_name': 'train2'}
bbb_martins_50
{'valid': 1.0, 'unique@1000': 0.9957805907172996, 'unique@10000': 0.9957805907172996, 'FCD/Test': 3.6841987379889503, 'SNN/Test': 0.5064036945397173, 'Frag/Test': 0.966294915338154, 'Scaf/Test': 0.5502519513284054, 'FCD/TestSF': 20.862980852147757, 'SNN/TestSF': 0.3738190557404242, 'Frag/TestSF': 0.7047772230048593, 'Scaf/TestSF': 0.02450489523

## Synthetic graphs 

In [20]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn','structural', 'auc_roc']
generators=['swingnn']
splits=['_25','_33','_40','_50']
experiments=[5,10,25,33,40,50,66,75,100]
metrics={}
for generator in ['swingnn']:
    for dataset_name in ['synthetic_1','synthetic_2']:
        for split in splits:
            for percent in experiments:
                print(dataset_name+split)
                train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
                generated_graphs, generated_targets= get_swingnn_graph_data(dataset_name,percent,split)
                splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
                metrics=evaluate(train1_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
                metrics['dataset']=dataset_name+split
                metrics['generator_name']=generator
                metrics['numer_of_epochs_trained']=(percent*5000)/100
                print(metrics)

synthetic_1_25
Now computing classifier based metrics
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Now computing structural based metrics
Now computing the auc_roc based  metric
0.7019 0.7620000000000001 0.42635 0.5492999999999999
{'activations_time': 0.021010875701904297, 'fid': 1992.0837783679606, 'fid_time': 0.04033160209655762, 'kid': 32062.93, 'kid_time': 4.84860897064209, 'precision': 0.01485148514851485, 'recall': 0.025, 'f1_pr': 0.018644188556302177, 'precision_time': 0.17827486991882324, 'recall_time': 0.17827486991882324, 'f1_pr_time': 0.17827486991882324, 'density': 0.0029702970297029703, 'coverage': 0.005, 'f1_dc': 0.0037373549602240336, 'density_time': 0.08560895919799805, 'coverage_time': 0.08560895919799805, 'f1_dc_time': 0.08560895919799805, 'mmd_rbf': 1.2035624831914902, 'mmd_rbf_t

In [26]:
structural_statistic=['WL', 'nspdk','degree','cluster', 'spectral']
device=torch.device('cpu')
metrics_type=['nn','structural', 'auc_roc']
generators=['train2']
splits=['_25','_33','_40','_50']
experiments=[5,10,25,33,40,50,66,75,100]
metrics={}
for generator in ['train2']:
    for dataset_name in ['synthetic_1','synthetic_2']:
        for split in splits:
            print(dataset_name+split)
            train1_graphs,train1_targets,train2_graphs,train2_targets,test_graphs,test_targets=get_graph_data(dataset_name+split, path=r'data/graphs/datasets')
            generated_graphs, generated_targets=train2_graphs,train2_targets
            splits_for_auc_roc=[train1_graphs , train1_targets,train2_graphs , train2_targets, test_graphs, test_targets, generated_graphs, generated_targets]
            metrics=evaluate(train1_graphs, generated_graphs, device,  metrics_type, structural_statistic,*splits_for_auc_roc)
            metrics['dataset']=dataset_name+split
            metrics['generator_name']=generator
            print(metrics)


synthetic_1_25
Now computing classifier based metrics
Now computing structural based metrics
Now computing the auc_roc based  metric
0.7019 0.7620000000000001 0.7061000000000001 0.7620000000000001
{'activations_time': 0.09806990623474121, 'fid': 0.22037899701926023, 'fid_time': 0.32657504081726074, 'kid': -0.002235651, 'kid_time': 0.1197969913482666, 'precision': 0.97, 'recall': 0.99, 'f1_pr': 0.9799079602248955, 'precision_time': 1.095655918121338, 'recall_time': 1.095655918121338, 'f1_pr_time': 1.095655918121338, 'density': 1.026, 'coverage': 0.98, 'f1_dc': 1.0024825875115901, 'density_time': 0.22144103050231934, 'coverage_time': 0.22144103050231934, 'f1_dc_time': 0.22144103050231934, 'mmd_rbf': 0.00850243540480733, 'mmd_rbf_time': 0.36985182762145996, 'mmd_linear': 0.03589476, 'mmd_linear_time': 0.098114013671875, 'wl_mmd': 0.01, 'wl_mmd_time': 0.24774694442749023, 'nspdk_mmd': 0.008445549078421655, 'nspdk_mmd_time': 4.805341005325317, 'degree_mmd': 0.00026336114711855707, 'degree_m